## How to create a closed-loop optimisation using the two objective Kursawe synthetic function as the benchmark function

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

### Setting up the variables and objectives for the synthetic function

The following table shows the ideal parameters to use for each multi-objective synthetic function

| Name | Maximise/Minimise | String | Number of variables | Number of objectives | Variable bounds | Objective bounds |
| --- | --- | --- | --- | --- | --- | --- |
| Kursawe | Minimise | "kursawe" | 3 | 2 | all [-5.0, 5.0] | approx obj1: [-20.0, -5.0], obj2: [-13.0, 23.0] |
| ZDT1 | Minimise | "zdt1" | 30 | 2 | all [0.0, 1.0] | approx obj1: [0.0, 1.0], obj2: [0.0, 7.5] |
| ZDT2 | Minimise | "zdt2" | 30 | 2 | all [0.0, 1.0] | approx obj1: [0.0, 1.00], obj2: [0.0, 8.0] |
| ZDT3 | Minimise | "zdt3" | 30 | 2 | all [0.0, 1.0] | approx obj1: [0.0, 1.0], obj2: [0.0, 7.5] |
| ZDT4 | Minimise | "zdt4" | 10 | 2 | var1: [0.0, 1.0], var2 to var10: [-10.0, 10.0] | approx obj1: [0.0, 1.0], obj2: [0.0, 900.0] |
| ZDT6 | Minimise | "zdt6" | 10 | 2 | all [0.0, 1.0] | approx obj1: [0.0, 1.0], obj2: [0.0, 10.0] |

The number of variable and objective objects need to be created according to the specified described in the table above.

The example below uses the Kursawe function.

In [ ]:
# Create the variable objects
var_list = VariablesList(
    [ContinuousVariable(name=f"var{n}", lower_bound=-5.0, upper_bound=5.0, units="") for n in range(1, 4)]
)

In [ ]:
# Create the objective objects
obj1 = RegressionObjective(name="Kursawe_obj1", obj_max_bool=False, lower_bound=-20.0, upper_bound=-5.0)
obj2 = RegressionObjective(name="Kursawe_obj2", obj_max_bool=False, lower_bound=-13.0, upper_bound=23.0)
obj_list = ObjectivesList([obj1, obj2])

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

### Setting up the benchmark

The `MultiObjectiveSyntheticBenchmark` class is instantiated with the string name of the synthetic function as the argument

In [ ]:
from nemo_bo.opt.benchmark import MultiObjectiveSyntheticBenchmark

# Instantiate the MultiObjectiveSyntheticBenchmark class to be the benchmark function
benchmark = MultiObjectiveSyntheticBenchmark("kursawe")

In [ ]:
# Set up the optimisation instance whilst passing benchmark function
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler, benchmark_func=benchmark)

### Starting the optimisation run with the benchmark function

When using a benchmark function in an optimisation, X and Y arrays do not need to be passed. The training set will be automatically created, where the number of samples is related to the sampler type chosen and the number of variables (`number of samples = (2 * number of variables) + 2`)

When the plot_progress keyword argument is True, a pareto plot will be created at every iteration.

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
optimisation_data = optimisation.run(number_of_iterations=50, plot_progress=True)